In [0]:
#!git clone https://github.com/JuanIsernGhosn/coral-classification

In [0]:
experiment = "coral-classification-deep-learning/experiments/vgg16_Keras_2/"

In [3]:
# Imports
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
import numpy
import math
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, Activation, merge
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Nadam
import keras.backend as K
from natsort import natsorted, ns
import keras
import PIL
from keras.regularizers import l2
from glob import glob
from keras.applications import vgg16
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [0]:
# Fijar las semillas
seed = 2032 # Semilla del numpy
tf.set_random_seed(seed)# Fijar semilla del keras/tensorflow

epochs = 100
batch_size = 32

model_name = 'model.h5'

In [0]:
# Train and test directories
train_dir = "coral-classification/coral_img/train/"
test_dir = "coral-classification/coral_img/Test_Mixed/"

In [6]:
# Classes
clases = sorted(os.listdir(train_dir))
print(clases)

x_train = np.array([cv2.imread(os.path.join(train_dir, cl, name)) for cl in clases
           for name in os.listdir(os.path.join(train_dir, cl))])
y_train = np.array([n for n, cl in enumerate(clases)
           for name in os.listdir(os.path.join(train_dir, cl))])

idx = np.random.permutation(len(x_train))
x_train, y_train = x_train[idx], y_train[idx]

test_files = natsorted(os.listdir(test_dir))

x_test = np.array([cv2.imread(os.path.join(test_dir, name)) 
                   for name in test_files])

['ACER', 'APAL', 'CNAT', 'DANT', 'DSTR', 'GORG', 'MALC', 'MCAV', 'MMEA', 'MONT', 'PALY', 'SPO', 'SSID', 'TUNI']


In [7]:

x_train = x_train.astype('float32')
x_train /= 255

x_test = x_test.astype('float32')
x_test /= 255

y_train = to_categorical(y_train, dtype=int)

print(x_train.shape)
print(len(clases))
print(y_train.shape)

(620, 256, 256, 3)
14
(620, 14)


In [0]:
datagen = ImageDataGenerator(
   # rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [9]:
# create the base pre-trained model
base_model = vgg16.VGG16(weights='imagenet',include_top=False, input_shape=(256,256,3))

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(14,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input, outputs=preds)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_average_pooling2d_1
20 dense_1
21 dense_2
22 dense_3
23 dense_4


In [0]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer="adam", 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0, patience=5)

In [13]:
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)


# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size, subset="training"),
                   steps_per_epoch=int(len(x_train)*0.8) / batch_size, epochs=epochs,
                   validation_data = datagen.flow(x_train, y_train, batch_size=batch_size, subset='validation'),
                   validation_steps = int(len(x_train)*0.2) / batch_size,
                   verbose=1, callbacks=[es])

model.save(model_name)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
16/15 [==============================] - 21s 1s/step - loss: 2.4475 - acc: 0.1679 - val_loss: 2.0683 - val_acc: 0.3468
Epoch 2/100
16/15 [==============================] - 8s 500ms/step - loss: 1.7529 - acc: 0.4415 - val_loss: 1.4248 - val_acc: 0.5081
Epoch 3/100
16/15 [==============================] - 11s 683ms/step - loss: 1.2947 - acc: 0.5703 - val_loss: 1.2786 - val_acc: 0.5565
Epoch 4/100
16/15 [==============================] - 11s 677ms/step - loss: 1.1619 - acc: 0.6190 - val_loss: 0.9604 - val_acc: 0.7177
Epoch 5/100
16/15 [==============================] - 11s 675ms/step - loss: 0.9667 - acc: 0.6704 - val_loss: 0.9174 - val_acc: 0.6532
Epoch 6/100
16/15 [==============================] - 11s 687ms/step - loss: 0.8029 - acc: 0.7345 - val_loss: 0.7757 - val_acc: 0.7661
Epoch 7/100
16/15 [==============================] - 11s 667ms/step - loss: 0.6776 - acc: 0.7772 - val_loss: 0.8332 - val_acc: 0.7339
Epoch 8/100
16/15 

In [14]:
predictions_test = model.predict(x_test, verbose=1)

data = pd.DataFrame()
data['Id'] = test_files
data['Category'] = predictions_test.argmax(axis=-1)


data.to_csv("envio.csv", index=False)

146/146 [==============================] - 4s 27ms/step


In [15]:
data

,Id,Category
0,aT1.jpg,0
1,aT2.jpg,0
2,aT5.jpg,0
3,aT17.jpg,0
4,aT21.jpg,1
5,aT31.jpg,3
6,aT33.jpg,7
7,aT34.jpg,0
8,aT47.jpg,0
9,aT55.jpg,0
